In [1]:
import pandas as pd
# To save model weights and features
import pickle
# models
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec
from model_config import *

In [2]:
start_train_set =pd.read_csv(analysis_direction_path_tweets + 'train_test_data/train_set.csv',  lineterminator='\n', error_bad_lines=False)
start_train_set.dropna(inplace=True)
start_test_set  = pd.read_csv(analysis_direction_path_tweets + 'train_test_data/test_set.csv',  lineterminator='\n', error_bad_lines=False)
start_test_set.dropna(inplace=True)

In [3]:
# look how the train set is shuffle
start_train_set.dropna()
print(start_train_set.info())
start_train_set.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12060 entries, 0 to 12059
Data columns (total 2 columns):
class         12060 non-null int64
tweet_text    12060 non-null object
dtypes: int64(1), object(1)
memory usage: 282.7+ KB
None


,class,tweet_text
0,0,استقدمو الصين وزراء ربعنا ماههم كفو
1,0,اكثر ما نستطيع فعله مقاطعه الصين باكبر قدر مم...
2,0,سمعنا_صوتك_ياهادي الحمله حملت سفير الصين علي ...
3,0,اوزيل حتي باشد فتراته سوءا كان يتحفنا بتمريرات...
4,0,لك بان الامريكان اقويا فقد كذب عزيزي قوه الامر...


In [4]:
# look how the test set is shuffle 
start_test_set.dropna()
print(start_test_set.info())
start_test_set.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3014 entries, 0 to 3014
Data columns (total 2 columns):
class         3014 non-null int64
tweet_text    3014 non-null object
dtypes: int64(1), object(1)
memory usage: 70.6+ KB
None


,class,tweet_text
0,1,ربنا احفظك واحميك واغطي واردك سالم غانم وكل اخ...
1,0,الرياض\n الصين تستبدل الالعاب الناريه والليز...
2,1,لو سمعحتوا فعلا الفايرس كورونا ياثر علي المنتج...
3,1,عاجل بكين تعلن حاله وفاه في العاصمه فيروس كو...
4,0,المطلوب التركيز علي ادامه زخم دعوات الانسحاب ا...


# Prepare Data for Features Engineering

In [5]:
tweets_train, target_train, tweets_test, target_test = split_train_test_set(start_train_set, start_test_set)

Number of Training Tweets is:  12060
Number of Tagets class in Training Tweets is:  12060
Number of Testing Tweets is:  3014
Number of Tagets class in Testing Tweets is:  3014


# The whole Data set

In [6]:
df_file = pd.read_csv(analysis_direction_path_tweets + 'train_test_data/last_classifed_tweets_to_model_on_over_15000_tweets.csv', lineterminator='\n', error_bad_lines=False)
df_file.dropna(inplace=True)
print(len(df_file))
print("The number of class 1 which represent this tweet talks about coronavirus: ", len(df_file[df_file['class'] == 1]))
print("The number of class 0 which represent this tweet not talks about coronavirus: ", len(df_file[df_file['class'] == 0]))
df_file.head()

15075
The number of class 1 which represent this tweet talks about coronavirus:  7267
The number of class 0 which represent this tweet not talks about coronavirus:  7808


,class,tweet_text
0,1,طب مسلمين الصين اللي بيموتو الفيرس ذنب برده ا...
1,1,علي ما اذكر معظم الفيروسات ما عدا الايبولا ك...
2,1,اميركا السبب لضرب الاقتصاد الصيني وتركيع الصين...
3,1,قلنا نقاطع الصين الناس قالت مش ممكن الان مق...
4,1,نصحيتين اقدمها لك 1 لا تتروش في البرد 2 اح...


# Load the model


In [7]:
word_to_vec_model = Word2Vec.load('models_weights/word2vec/word2vec_features.bin')

In [8]:
word_to_vec_model.most_similar('كورونا')

/home/abdelrahman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('كورونا_الجديد', 0.8745729923248291),
 ('الجديد', 0.8651182651519775),
 ('فيروس_كورونا', 0.8538630604743958),
 ('مخاوف', 0.845136284828186),
 ('القاتل', 0.8410530090332031),
 ('عالمي', 0.8361520767211914),
 ('حدودها', 0.8341037034988403),
 ('تاثير', 0.8300197720527649),
 ('غامض', 0.8267508149147034),
 ('لمنع', 0.8252075910568237)]

In [9]:
text_list = list(df_file['tweet_text'])
text_list = [i.split() for i in text_list] # split each review to list of words
max_len_str = max([len(tweet) for tweet in text_list])
print(max_len_str)

63


In [10]:
tweets_train = list(tweets_train)
tweets_test = list(tweets_test)

# Before
print(tweets_train[:3])
print(tweets_test[:3])

tweets_train_splited = [tweet.split() for tweet in tweets_train] 
tweets_test_splited = [tweet.split() for tweet in tweets_test] 

# After
print(tweets_train_splited[:3])
print(tweets_test_splited[:3])

['استقدمو الصين وزراء ربعنا ماههم كفو', ' اكثر ما نستطيع فعله مقاطعه الصين باكبر قدر ممكن والتحريض علي المقاطعه افراد وشركات لعلها تتطور لحكومات   ', ' سمعنا_صوتك_ياهادي الحمله حملت سفير الصين علي الذهاب ابن منصور يتفاقده عاده موجود ويبدو لقي الضاءعه حقنا بالله خرجوه ولو بكيس قمامه وبعدين نتفاهم ههههههه   ']
['ربنا احفظك واحميك واغطي واردك سالم غانم وكل اخوانا الفي الصين   لاينسانا ولا ينساكم   اعمل حسابك   شا الله متلمي علي شباب سودانين', ' الرياض\n  الصين تستبدل الالعاب الناريه والليزر بطاءرات الدروز للاحتفال براس السنه لاول مره \n  تطور مذهل ربما ينهي عصر الالعاب الناريه  ', 'لو سمعحتوا فعلا الفايرس كورونا ياثر علي المنتجات        توني طالبه منكم وقالوا لاحد يطلب عشان الفايرس اللي بالصين  ']
[['استقدمو', 'الصين', 'وزراء', 'ربعنا', 'ماههم', 'كفو'], ['اكثر', 'ما', 'نستطيع', 'فعله', 'مقاطعه', 'الصين', 'باكبر', 'قدر', 'ممكن', 'والتحريض', 'علي', 'المقاطعه', 'افراد', 'وشركات', 'لعلها', 'تتطور', 'لحكومات'], ['سمعنا_صوتك_ياهادي', 'الحمله', 'حملت', 'سفير', 'الصين', 'علي', 'الذهاب', 'ابن', 'م

In [11]:
number_of_features = 300
train_embedding_matrix = word_2_vec_matrix(tweets_train_splited, word_to_vec_model, number_of_features, max_len_str)

The shape of matrix (12060, 18900)


In [12]:
param_grid = {'n_estimators':[30, 50, 70] ,'max_features':  ['auto', 'log2']}


RandomForestClassifier_model1 = RandomForestClassifier()
RanForCla_model1_model_search1 = grid_search(RandomForestClassifier_model1, param_grid,
                                             train_embedding_matrix, target_train)

In [14]:
RanForCla_model1_model_search1.best_estimator_

RandomForestClassifier(n_estimators=70)

In [15]:
check_search_params(RanForCla_model1_model_search1)

0.8483416252072968 {'max_features': 'auto', 'n_estimators': 30}
0.8510779436152571 {'max_features': 'auto', 'n_estimators': 50}
0.8529021558872305 {'max_features': 'auto', 'n_estimators': 70}
0.8194859038142621 {'max_features': 'log2', 'n_estimators': 30}
0.8254560530679933 {'max_features': 'log2', 'n_estimators': 50}
0.8316749585406301 {'max_features': 'log2', 'n_estimators': 70}


True

In [16]:
test_embedding_matrix = word_2_vec_matrix(tweets_test_splited, word_to_vec_model, number_of_features, max_len_str)

The shape of matrix (3014, 18900)


In [18]:
RanForCla_model1 = RandomForestClassifier(n_estimators=70, max_features='auto')
RanForCla_model1.fit(train_embedding_matrix, target_train)

model_predict_train__test_data(RanForCla_model1, train_embedding_matrix, target_train,
                              test_embedding_matrix, target_test)

===================== Training Result =====================
F1 score of our training data is:  0.9994195688225539
Evalution Matrix of training data is 
 [[6245    1]
 [   6 5808]]
===================== Testing Result =====================
F1 score of our Testing data is:  0.852023888520239
Evalution Matrix of Testing data is 
 [[1441  120]
 [ 326 1127]]


True

In [20]:
RanForCla_model2 = RandomForestClassifier(n_estimators=70, max_features='log2')
RanForCla_model2.fit(train_embedding_matrix, target_train)

model_predict_train__test_data(RanForCla_model2, train_embedding_matrix, target_train,
                              test_embedding_matrix, target_test)

===================== Training Result =====================
F1 score of our training data is:  0.9994195688225539
Evalution Matrix of training data is 
 [[6243    3]
 [   4 5810]]
===================== Testing Result =====================
F1 score of our Testing data is:  0.8354346383543464
Evalution Matrix of Testing data is 
 [[1452  109]
 [ 387 1066]]


True

In [22]:
pickle.dump(RanForCla_model1, open('models_weights/random_forset/word2vec_RandomForestClassifier_99tr_85ts.sav', 'wb'))